In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

import numbers
from sklearn import model_selection


In [2]:
# Aquí, a futuro, habría que añadir los datos del Nasdaq, oro y/o Ethereum para ver los resultados
data_for_use = pd.read_csv('./bitcoin_data_for_use_v2.csv', index_col=0)
data_gold = pd.read_csv('./gold_data_for_use.csv', index_col=0)
data_nasdaq = pd.read_csv('./nasdaq_data_for_use.csv', index_col=0)

data_for_use_w_gold = data_for_use.merge(data_gold, left_index=True, right_index=True)
data_for_use_w_nasdaq = data_for_use.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_w_all = data_for_use_w_gold.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_basic = data_for_use.drop(labels=[
                    'varV0','varV1','varV2','varV3','varV4','varV5','varV6','varV7','varV8','varV9',
                    'varV10','varV11','varV12','varV13','varV14','varV15','varV16','varV17','varV18','varV19',
                    'varV20','varV21','varV22','varV23','varV24','varV25','varV26','varV27','varV28','varV29'
                    ], axis=1)

data_for_use_bone_deep = data_for_use_basic.drop(labels=['varPm93',
                    'varPm123','varPm153','varPm183','varPm213','varPm243','varPm273','varPm303','varPm333',
                    'varPs30','varPs37','varPs44','varPs51','varPs58','varPs65','varPs72','varPs79','varPs86'
                    ], axis=1)


In [3]:
data_for_use_basic

,varP0,varP1,varP2,varP3,varP4,varP5,varP6,varP7,varP8,varP9,...,varPm123,varPm153,varPm183,varPm213,varPm243,varPm273,varPm303,varPm333,dia,subida
2016-01-01,0.008749,0.009252,-0.014696,0.025348,-0.001287,0.013298,-0.084229,0.001468,0.028445,0.013352,...,-0.183043,0.102536,0.131128,-0.060556,-0.054907,-0.068731,0.146342,-0.152457,Friday,False
2016-01-02,-0.002063,0.008749,0.009252,-0.014696,0.025348,-0.001287,0.013298,-0.084229,0.001468,0.028445,...,-0.192818,0.102514,0.134863,-0.054992,-0.057860,-0.081400,0.215208,-0.139772,Saturday,False
2016-01-03,-0.007907,-0.002063,0.008749,0.009252,-0.014696,0.025348,-0.001287,0.013298,-0.084229,0.001468,...,-0.184701,0.077969,0.162987,-0.049962,-0.093926,-0.044459,0.202201,-0.173499,Sunday,True
2016-01-04,0.007163,-0.007907,-0.002063,0.008749,0.009252,-0.014696,0.025348,-0.001287,0.013298,-0.084229,...,-0.203476,0.048931,0.208760,-0.021020,-0.100629,-0.075179,0.272441,-0.241372,Monday,False
2016-01-05,-0.002611,0.007163,-0.007907,-0.002063,0.008749,0.009252,-0.014696,0.025348,-0.001287,0.013298,...,-0.182999,0.047772,0.192408,-0.049361,-0.062588,-0.077178,0.234350,-0.244859,Tuesday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-21,-0.050185,-0.093449,-0.021089,0.021386,-0.010795,-0.008157,0.023015,0.047359,-0.023884,0.019066,...,-0.307969,-0.011426,-0.027993,0.819894,0.296122,0.294667,0.420558,0.264591,Tuesday,True
2021-09-22,0.070793,-0.050185,-0.093449,-0.021089,0.021386,-0.010795,-0.008157,0.023015,0.047359,-0.023884,...,-0.274826,-0.054379,-0.048679,0.743330,0.420131,0.265584,0.400979,0.219860,Wednesday,True
2021-09-23,0.030306,0.070793,-0.050185,-0.093449,-0.021089,0.021386,-0.010795,-0.008157,0.023015,0.047359,...,-0.319474,-0.031845,-0.026437,0.690406,0.351016,0.242234,0.466289,0.217606,Thursday,False
2021-09-24,-0.045781,0.030306,0.070793,-0.050185,-0.093449,-0.021089,0.021386,-0.010795,-0.008157,0.023015,...,-0.226667,-0.031976,0.058981,0.512089,0.309128,0.316711,0.432640,0.215800,Friday,False


In [4]:
#aqui se marca lo que se va a usar de verdad para el resto del programa
data_for_use = data_for_use_basic

In [5]:
# Partimos los datos en train y test por fechas, sin que sea aleatorio, que es como funcionaría
# en la realidad el modelo
data_no_test, data_test = train_test_split(data_for_use, shuffle=False, test_size=0.2)

# Partimos los datos de train nuevamente en validation y train
data_train, data_valid = train_test_split(data_no_test, shuffle=False, test_size=0.2)

In [6]:
# Separamos la variable que queremos predecir
X_train=data_train.drop(labels=['subida'], axis=1)
y_train=data_train['subida']

X_valid=data_valid.drop(labels=['subida'], axis=1)
y_valid=data_valid['subida']

X_test=data_test.drop(labels=['subida'], axis=1)
y_test=data_test['subida']

In [7]:
# Miramos cuántos días sube el bitcoin sobre el total en cada uno de los segmentos
# esto nos servirá para hacernos una idea de cómo de buenos son los resultados de los modelos
y_train.sum()/y_train.size

0.5686567164179105

In [8]:
y_valid.sum()/y_valid.size

0.4851190476190476

In [9]:
y_test.sum()/y_test.size

0.5513126491646778

In [10]:
#Tenemos que hacer one-hot encoding sobre la columna "dia"
#Sobre el resto aplicamos el standardscaler

number_columns = data_for_use.select_dtypes('number').columns

t=[
    ('dia', 
    OneHotEncoder(handle_unknown='ignore'),
    ['dia']),
    ('scaler', StandardScaler(),number_columns)
    ]

ct = ColumnTransformer(transformers=t, remainder='passthrough')

#data_for_use_t=ct.fit_transform(data_for_use)

#X_train=ct.fit_transform(X_train)
#X_valid=ct.transform(X_valid)
#X_test=ct.transform(X_test)

In [11]:
def walk_forward_validation (model, X_train_wfv, y_train_wfv, X_test_wfv, y_test_wfv):
    y_pred_wfv = list()
    for i in range(len(y_test_wfv)):
        X_train_wfv_ct=ct.fit_transform(X_train_wfv)
        model.fit(X_train_wfv_ct, y_train_wfv)
        X_test_wfv_ct=ct.transform(X_test_wfv)
        y_pred_next = model.predict(X_test_wfv_ct[i:i+1])
        y_pred_wfv.append(y_pred_next[0])
        X_train_wfv=X_train_wfv.append(X_test_wfv[i:i+1])
        y_train_wfv=y_train_wfv.append(pd.Series(y_test_wfv[i]))
    
    return metrics.confusion_matrix(y_test_wfv, y_pred_wfv), metrics.f1_score(y_test_wfv,y_pred_wfv), metrics.accuracy_score(y_test_wfv,y_pred_wfv)
        
        

In [12]:
lr = LogisticRegression(penalty='l2', solver='lbfgs')

X_train_c=X_train.copy()
y_train_c=y_train.copy()
X_valid_c=X_valid.copy()
y_valid_c=y_valid.copy()


conf_mat, f1sc, accsc = walk_forward_validation(lr, X_train_c, y_train_c, X_valid_c, y_valid_c)

In [13]:
print(conf_mat, f1sc, accsc)

[[ 40 133]
 [ 32 131]] 0.613583138173302 0.5089285714285714


In [14]:
for vecinos in range(1,31):
    kn = KNeighborsClassifier(n_neighbors=vecinos)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(kn, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Numero de vecinos aplicado: " + str(vecinos))
    print(conf_mat, f1sc, accsc)

Numero de vecinos aplicado: 1
[[ 59 114]
 [ 61 102]] 0.5382585751978891 0.4791666666666667
Numero de vecinos aplicado: 2
[[102  71]
 [ 98  65]] 0.43478260869565216 0.49702380952380953
Numero de vecinos aplicado: 3
[[ 55 118]
 [ 53 110]] 0.5626598465473145 0.49107142857142855
Numero de vecinos aplicado: 4
[[80 93]
 [76 87]] 0.5072886297376094 0.49702380952380953
Numero de vecinos aplicado: 5
[[ 46 127]
 [ 43 120]] 0.5853658536585366 0.49404761904761907
Numero de vecinos aplicado: 6
[[ 66 107]
 [ 61 102]] 0.5483870967741935 0.5
Numero de vecinos aplicado: 7
[[ 33 140]
 [ 31 132]] 0.6068965517241378 0.49107142857142855
Numero de vecinos aplicado: 8
[[ 53 120]
 [ 46 117]] 0.585 0.5059523809523809
Numero de vecinos aplicado: 9
[[ 30 143]
 [ 23 140]] 0.6278026905829596 0.5059523809523809
Numero de vecinos aplicado: 10
[[ 42 131]
 [ 40 123]] 0.5899280575539568 0.49107142857142855
Numero de vecinos aplicado: 11
[[ 22 151]
 [ 19 144]] 0.62882096069869 0.49404761904761907
Numero de vecinos aplic

In [ ]:
for profund in range(1,31):
    dt = DecisionTreeClassifier(max_depth=profund)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(dt, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Maxima profundidad aplicada: " + str(profund))
    print(conf_mat, f1sc, accsc)

Maxima profundidad aplicada: 1
[[ 15 158]
 [ 10 153]] 0.6455696202531646 0.5
Maxima profundidad aplicada: 2
[[ 23 150]
 [ 22 141]] 0.6211453744493393 0.4880952380952381
Maxima profundidad aplicada: 3
[[ 33 140]
 [ 30 133]] 0.6100917431192661 0.49404761904761907
Maxima profundidad aplicada: 4
[[ 31 142]
 [ 30 133]] 0.6073059360730593 0.4880952380952381
Maxima profundidad aplicada: 5
[[ 33 140]
 [ 35 128]] 0.5939675174013922 0.4791666666666667


In [ ]:
for estimadores in range(1,21):
    adab = AdaBoostClassifier(n_estimators=estimadores)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(adab, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Estimadores aplicados: " + str(estimadores))
    print(conf_mat, f1sc, accsc)

In [ ]:
dt = DecisionTreeClassifier(max_depth=9)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
dt.fit(X_train_c,y_train_c)
y_predict=dt.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

In [ ]:
dt = DecisionTreeClassifier(max_depth=16)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
dt.fit(X_train_c,y_train_c)
y_predict=dt.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

In [ ]:
adab = AdaBoostClassifier(n_estimators=19)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
adab.fit(X_train_c,y_train_c)
y_predict=adab.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

In [ ]:
kn = KNeighborsClassifier(n_neighbors=3)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
kn.fit(X_train_c,y_train_c)
y_predict=kn.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

In [47]:
kn = KNeighborsClassifier(n_neighbors=15)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
kn.fit(X_train_c,y_train_c)
y_predict=kn.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[239, 512],
        [134, 791]]),
 0.7100538599640933,
 0.6145584725536993)

In [13]:
#Este es el código para ver cómo funciona el modelo de verdad (este es el resultado del basic)

dt = DecisionTreeClassifier(max_depth=14)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)
X_test_c=X_test.copy()
y_test_c=y_test.copy()

conf_mat, f1sc, accsc = walk_forward_validation(dt, X_train_c, y_train_c, X_test_c, y_test_c)

print(conf_mat, f1sc, accsc)

[[ 81 107]
 [ 92 139]] 0.5828092243186582 0.5250596658711217
